From Tensor SkFlow: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/skflow/iris.py

## Import

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from sklearn import metrics, cross_validation

from tensorflow.contrib import learn

import chainer.functions as F
import chainer.links as L
from chainer import optimizers, Chain
from commonml.sklearn import ChainerEstimator, SoftmaxCrossEntropyClassifier

import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = 20

## Load dataset.

In [ ]:
iris = learn.datasets.load_dataset('iris')
X_train, X_test, y_train, y_test = cross_validation.train_test_split(iris.data, iris.target,
    test_size=0.2, random_state=42)

## Build 3 layer DNN with 10, 20, 10 units respectively.

In [ ]:
class Model(Chain):

    def __init__(self, in_size):
        super(Model, self).__init__(l1=L.Linear(in_size, 10),
                                    l2=L.Linear(10, 20),
                                    l3=L.Linear(20, 10),
                                    l4=L.Linear(10, 3),
                                   )

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        h3 = F.relu(self.l3(h2))
        h4 = self.l4(h3)
        return h4

classifier = ChainerEstimator(model=SoftmaxCrossEntropyClassifier(Model(X_train.shape[1])),
                              optimizer=optimizers.AdaGrad(lr=0.1),
                              batch_size=100,
                              gpu=0,
                              n_epoch=100)

## Fit and predict.

In [ ]:
classifier.fit(X_train, y_train)
score = metrics.accuracy_score(y_test, classifier.predict(X_test))
print('Accuracy: {0:f}'.format(score))